In [1]:
import csv
from cobra.io import read_sbml_model, write_sbml_model
from copy import copy
from dotenv import find_dotenv
import os
from os.path import dirname

In [2]:
dotenv_path = find_dotenv()
REPO_PATH = dirname(dotenv_path)
MODEL_PATH = "models/yeast-GEM.xml"

In [3]:
#model = read_sbml_model(MODEL_PATH) # Lu, H., Li, F., Sánchez, B.J. et al. A consensus S. cerevisiae metabolic model Yeast8 and its ecosystem for comprehensively probing cellular metabolism. Nat Commun 10, 3586 (2019). https://doi.org/10.1038/s41467-019-11581-3

In [4]:
def read_yeast_model(make_bigg_compliant=True):
    """This function is made by researchers  at 
    Chalmers University of Technology Department of Biology
    and Biological Engineering Division of Systems and 
    Synthetic Biology in therir project titled 
    "The consensus GEM for Saccharomyces cerevisiae"
    which can be found at: https://github.com/SysBioChalmers/yeast-GEM
        
    The functions reads the SBML file of the yeast model using COBRA, 
    and imposes changes to the annotations so that the model is BiGG compliant.

    Parameters
    ----------
    make_bigg_compliant : bool, optional
        Whether the model should be initialized with BiGG compliance or not.
        If false, the original ids/names/compartments will be used instead.

    Returns
    -------
    cobra.core.Model
    """

    # Load model:
    model = read_sbml_model(MODEL_PATH)

    # Check if already BiGG compliant:
    is_bigg_compliant = "x" in model.compartments

    # Convert to BiGG compliant if not already:
    if not is_bigg_compliant and make_bigg_compliant:
        # Load met/rxn dictionaries:
        def load_bigg_dict(bigg_file_path):
            bigg_dict = {}
            with open(bigg_file_path) as bigg_file:
                bigg_reader = csv.reader(bigg_file, delimiter=',')
                for row in bigg_reader:
                    bigg_dict[row[0]] = row[1]
            return bigg_dict
        data_path = f"{REPO_PATH}/data/databases"
        met_bigg_dict = load_bigg_dict(f"data/databases/BiGGmetDictionary_newIDs.csv")
        rxn_bigg_dict = load_bigg_dict(f"data/databases/BiGGrxnDictionary_newIDs.csv")

        # Function for adding unique ids to the model:
        def add_new_id(model_element, new_id):
            original_id = copy(new_id)
            id_assigned = False
            copy_number = 1
            while not id_assigned:
                try:
                    if hasattr(model_element, "compartment"):  # metabolites
                        model_element.id = f"{new_id}_{model_element.compartment}"
                    else:  # reactions
                        model_element.id = new_id
                    id_assigned = True
                except ValueError:
                    new_id = f"{original_id}_copy{str(copy_number)}"
                    copy_number += 1

        # Metabolite changes:
        comp_dic = {"er":"r", "erm":"rm", "p":"x"}
        for met in model.metabolites:
            # Save original id in notes:
            met.notes["Original ID"] = met.id
            # Change name to not include compartment at the end:
            met.name = met.name.replace(f" [{model.compartments[met.compartment]}]", "")
            # Change compartment info:
            if met.compartment in comp_dic:
                met.compartment = comp_dic[met.compartment]
            # Update id with BiGG information:
            if "bigg.metabolite" in met.annotation:
                add_new_id(met, met.annotation['bigg.metabolite'])
            elif met.id in met_bigg_dict:
                add_new_id(met, met_bigg_dict[met.id])
            else:
                met.id = met.id.replace(f"[{met.compartment}]", f"_{met.compartment}")

        # Compartment changes:
        comps = model.compartments
        comps["r"] = "endoplasmic reticulum"
        comps["rm"] = "endoplasmic reticulum membrane"
        comps["x"] = "peroxisome"
        model.compartments = comps

        # Reaction changes:
        for rxn in model.reactions:
            # Update id with BiGG information:
            if "bigg.reaction" in rxn.annotation:
                rxn.notes["Original ID"] = rxn.id
                add_new_id(rxn, rxn.annotation['bigg.reaction'])
            elif rxn.id in rxn_bigg_dict:
                rxn.notes["Original ID"] = rxn.id
                add_new_id(rxn, rxn_bigg_dict[rxn.id])

    return model

In [5]:
model = read_yeast_model() # loading

In [6]:
def write_yeast_model(model):
    """Writes the SBML file of the yeast model using COBRA.

    Parameters
    ----------
    model : cobra.core.Model
        Yeast model to be written
    """
    write_sbml_model(model, "model_GEM_BiGG.xml")

In [7]:
write_yeast_model(model)   # saving

# Manipulations

In [8]:
model.medium

{'EX_nh4_e': 1000.0,
 'EX_glc__D_e': 1.0,
 'EX_h_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0}

In [9]:
media_comp = list(model.medium)
for i in range (0, len(media_comp)):
    print(model.medium[media_comp[i]], model.reactions.get_by_id(media_comp[i]).name)
    if 'glucose' in model.reactions.get_by_id(media_comp[i]).name:
        carbon_exchange_id = media_comp[i]
        glucose_id = model.reactions.get_by_id(carbon_exchange_id).reaction[:9] # carbon source ID

1000.0 ammonium exchange
1.0 D-glucose exchange
1000.0 H+ exchange
1000.0 iron(2+) exchange
1000.0 oxygen exchange
1000.0 phosphate exchange
1000.0 potassium exchange
1000.0 sodium exchange
1000.0 sulphate exchange
1000.0 water exchange
1000.0 chloride exchange
1000.0 Cu2(+) exchange
1000.0 Mn(2+) exchange
1000.0 Zn(2+) exchange
1000.0 Mg(2+) exchange
1000.0 Ca(2+) exchange
glc__D_e 


In [10]:
print("The GEM Saccharomyces cerevisiae model contains "+str(len(model.genes))+" genes, "+str(len(model.metabolites))+" metabolites and "+str(len(model.reactions))+" reactions.")

The GEM Saccharomyces cerevisiae model contains 1150 genes, 2742 metabolites and 4058 reactions.


In [11]:
print(model.compartments)

{'ce': 'cell envelope', 'c': 'cytoplasm', 'e': 'extracellular', 'm': 'mitochondrion', 'n': 'nucleus', 'x': 'peroxisome', 'r': 'endoplasmic reticulum', 'g': 'Golgi', 'lp': 'lipid particle', 'v': 'vacuole', 'rm': 'endoplasmic reticulum membrane', 'vm': 'vacuolar membrane', 'gm': 'Golgi membrane', 'mm': 'mitochondrial membrane'}


In [12]:
for metabolite in model.metabolites.query('glucose', 'name'):
    print(metabolite.name,metabolite.id)

D-glucose glc__D_c
D-glucose glc__D_e
D-glucose glc__D_v
D-glucose 1-phosphate g1p_c
D-glucose 6-phosphate g6p_c
UDP-D-glucose udpg_c
D-glucose glc__D_r
2-deoxy-D-glucose 6-phosphate 2doxg6p_c
2-deoxy-D-glucose 2dglc_c
UDP-D-glucose udpg_r
D-glucose 1-phosphate g1p_e
D-glucose 6-phosphate g6p_e


In [13]:
print("The growth rate of GEM yeast is: "+str(model.optimize().objective_value))
biomass_id = str(model.objective).split(' ')[0].split('*')[1]

The growth rate of GEM yeast is: 0.08374778664999834


In [14]:
for reactions in model.reactions.query('ferrochelatase', 'name'):
    print(reactions.name,reactions.id)
    ferrochelatase_id = str(reactions.id)
model.reactions.get_by_id(ferrochelatase_id)

ferrochelatase FCLTm


Reaction identifier,FCLTm
Name,ferrochelatase
Memory address,0x020936e1d588
Stoichiometry,fe2_m + ppp9_m --> 2.0 h_m + pheme_m iron(2+) + protoporphyrin --> 2.0 H+ + ferroheme b
GPR,YOR176W
Lower bound,0.0
Upper bound,1000.0


In [15]:
medium = model.medium
with model:
    solution = model.optimize()
    print("GEM yeast maximum theoretical biomass productivity:", solution.fluxes[biomass_id], '/h')
    model.medium = medium
    model.objective = model.reactions.get_by_id(ferrochelatase_id)
    heme_production = model.optimize().objective_value
    print("GEM yeast maximum theoretical productivity of heme ", heme_production, '[mmol gDW^-1 h^-1]')

GEM yeast maximum theoretical biomass productivity: 0.08374778664999834 /h
GEM yeast maximum theoretical productivity of heme  8.374770489917409e-08 [mmol gDW^-1 h^-1]


## Knockouts

In [16]:
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
from cameo.strain_design import OptGene
from cobra import Reaction, Metabolite

### OptGene: Employs a genetic algorithm to find combination of gene deletions.

In [17]:
optgene = OptGene(model) # ACS Synth. Biol. 2018, 7, 4, 1163–1166 Publication Date:March 20, 2018 https://doi.org/10.1021/acssynbio.7b00423

In [10]:
result = optgene.run(target=model.reactions.get_by_id(ferrochelatase_id),
                     biomass=model.reactions.get_by_id(biomass_id),
                     substrate=model.metabolites.get_by_id(glucose_id),
                     max_evaluations=2,
                     plot=False)
result

NameError: name 'optgene' is not defined

In [ ]:
result.plot(0)

In [ ]:
result.display_on_map(0, "iMM904.Central carbon metabolism")

### OptReg: High-throughput calculations of knock-outs, up- or downregulations.

MEWpy: https://github.com/BioSystemsUM/mewpy

Vítor Pereira, Fernando Cruz, Miguel Rocha, MEWpy: a computational strain optimization workbench in Python, Bioinformatics, 2021;, btab013, https://doi.org/10.1093/bioinformatics/btab013

In [9]:
from cameo.strain_design import OptReg

ImportError: cannot import name 'OptForce' from 'cameo.strain_design' (C:\Users\andre\anaconda3\envs\gem\lib\site-packages\cameo\strain_design\__init__.py)

In [ ]:
result_reg = optgene.run(target=model.reactions.get_by_id(ferrochelatase_id),
                     biomass=model.reactions.get_by_id(biomass_id),
                     substrate=model.metabolites.get_by_id(glucose_id),
                     max_evaluations=2,
                     plot=False)
result_reg

### OptForce: High-throughput calculations of knock-outs, up- or downregulations. Can include MFA data, and model based on a target production.

In [ ]:
from cameo.strain_design import OptForce

In [ ]:
result_force = optforce.run(target=model.reactions.get_by_id(ferrochelatase_id),
                     biomass=model.reactions.get_by_id(biomass_id),
                     substrate=model.metabolites.get_by_id(glucose_id),
                     max_evaluations=2,
                     plot=False)
result_force

### Rationale-based genetic modifications

#### Up-regulation

HEM2	YGL040C,
HEM3	YDL205C,
HEM13	YDR044W,
HEM15	YOR176W

#### Knockout

HAP1	YLR256W

#### Downregulation
YFH1	YDL120W

#### Maybe interesting
CCC1	YLR220W	Transport af heme - into vacule
PUG1	YER185W	Export af heme - ud af cellen

In [13]:
for gene in model.genes.query('YFH1', 'name'):
    print(gene.name,gene.id)

YFH1 YDL120W
